In [112]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_community.vectorstores import Chroma

In [113]:
import os
import getpass
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

In [114]:
api_key = os.environ.get("OPENAI_API_KEY")

In [115]:
loader = PyPDFLoader("Whiplash.pdf")
doc = loader.load()

In [116]:
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
chunks = splitter.split_documents(doc)

In [117]:
embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 310df597-4f8d-419b-b81a-562f42022364)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


In [118]:
vectorstore = Chroma.from_documents(chunks, embeddings, collection_name = "whiplash_rag")
vectorstore.persist()
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 4})

In [119]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7, openai_api_key = api_key)

In [120]:
def call_model(state: MessagesState, llm, retriever):
    query = state["messages"][-1].content  # user message
    context_docs = vectorstore.similarity_search(query, k=4)
    context = "\n\n".join([doc.page_content for doc in context_docs])

    prompt = f"Context:\n{context}\n\nUser question: {query}\n\nAnswer:"
    response = llm.invoke(prompt)

    text = response.content if hasattr(response, "content") else str(response)
    
    return {"messages": state["messages"] + [HumanMessage(content=text)]}


In [121]:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", lambda state: call_model(state, llm, retriever))

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [122]:
message_history = []

query = "What does the movie Whiplash say about jazz?"
message_history.append(HumanMessage(content=query))


output = app.invoke({"messages": message_history}, config)

message_history.append(output["messages"][-1])
output["messages"][-1].pretty_print()

================================ Human Message =================================

The movie "Whiplash" presents a complex and intense portrayal of jazz, highlighting both its artistic demands and the rigorous discipline required to excel in it. Here are some key themes regarding jazz depicted in the film:

1. **Technical Mastery**: The film emphasizes the technical skill and precision needed to perform jazz at a high level. The character Andrew, a young drummer, faces immense pressure to master challenging compositions, illustrating the rigorous practice and dedication required in jazz music.

2. **Creativity vs. Conformity**: "Whiplash" explores the tension between individual creativity and adherence to strict musical standards. While jazz is often associated with improvisation and personal expression, the film shows how Andrew struggles to balance these elements with the demanding expectations of his conductor, Fletcher.

3. **Pressure and Sacrifice**: The film delves into the psycho

In [123]:
query = "What does it say about ambition?"
message_history.append(HumanMessage(content=query))

output = app.invoke({"messages": message_history}, config)
message_history.append(output["messages"][-1])
output["messages"][-1].pretty_print()

================================ Human Message =================================

The dialogue suggests that ambition can be a complex and evolving journey. Fletcher acknowledges that many young people, like Andrew, may not have a clear direction or specific passion at the beginning of their careers. This reflects a common reality that finding one’s calling often takes time and experimentation. Fletcher’s reference to his father’s varied experiences underscores the idea that ambition is not always linear; it can involve trying out different paths and learning from those experiences. Additionally, the caution about listening to advice from those with potential ulterior motives highlights the importance of discernment in pursuing ambition—suggesting that while external perspectives can be valuable, one should be wary of influences that may not have their best interests at heart. Overall, the conversation portrays ambition as a personal and often gradual process of self-discovery, shaped 

In [126]:
query = "How do these two concepts correlate"
message_history.append(HumanMessage(content=query))

output = app.invoke({"messages": message_history}, config)
message_history.append(output["messages"][-1])
output["messages"][-1].pretty_print()

================================ Human Message =================================

The two concepts correlate through the theme of personal ambition versus relational commitment. Andrew is expressing his concern that pursuing his passion for drumming will inevitably lead to a lack of time and attention for his relationship with Nicole. He believes that as he dedicates more of himself to his craft, the dynamic of their relationship will shift, causing Nicole to feel neglected and potentially resentful. 

This conversation highlights the struggle many individuals face when trying to balance personal goals with the needs of a partner. Andrew’s decision to articulate his feelings suggests an awareness of the potential negative impact of his ambition on their relationship, indicating a desire to be honest and prevent future conflict. In essence, it reflects the broader conflict between individual aspirations and the demands of romantic relationships, emphasizing the difficulty of maintaining

In [125]:
'''config = {"configurable": {"thread_id": "abc123"}}
query = "What does the movie Whiplash say about jazz?"
input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()'''


'config = {"configurable": {"thread_id": "abc123"}}\nquery = "What does the movie Whiplash say about jazz?"\ninput_messages = [HumanMessage(query)]\noutput = app.invoke({"messages": input_messages}, config)\noutput["messages"][-1].pretty_print()'